In [1]:
import bs4
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [2]:
DRIVER_PATH = "C:\Chromedriver\chromedriver.exe"

In [3]:
options = Options()
options.headless = False

In [4]:
driver = webdriver.Chrome(service=Service(DRIVER_PATH),options=options)

In [5]:
narutoShippuden = {
    "S1" : "https://episodes.animeflix.org.in/archives/6427",
    "S2" : "https://episodes.animeflix.org.in/archives/6428",
    "S3" : "https://episodes.animeflix.org.in/archives/6429",
    "S4" : "https://episodes.animeflix.org.in/archives/6430",
    "S5" : "https://episodes.animeflix.org.in/archives/6431",
    "S6" : "https://episodes.animeflix.org.in/archives/6432",
    "S7" : "https://episodes.animeflix.org.in/archives/6433",
    "S8" : "https://episodes.animeflix.org.in/archives/6434",
    "S9" : "https://episodes.animeflix.org.in/archives/6435",
    "S10" : "https://episodes.animeflix.org.in/archives/6448",
    "S11" : "https://episodes.animeflix.org.in/archives/6449",
    "S12" : "https://episodes.animeflix.org.in/archives/6445",
    "S13" : "https://episodes.animeflix.org.in/archives/6446",
    "S14" : "https://episodes.animeflix.org.in/archives/6447",
    "S15" : "https://episodes.animeflix.org.in/archives/6454",
    "S16" : "https://episodes.animeflix.org.in/archives/6455",
    "S17" : "https://episodes.animeflix.org.in/archives/6456",
    "S18" : "https://episodes.animeflix.org.in/archives/6457",
    "S19" : "https://episodes.animeflix.org.in/archives/6458",
    "S20.1" : "https://episodes.animeflix.org.in/archives/6459",
    "S20.2" : "https://episodes.animeflix.org.in/archives/6460",
    "S21" : "https://episodes.animeflix.org.in/archives/6668"
    }

In [6]:
Season = "S1"
driver.get(narutoShippuden[Season])

In [7]:
def getLinks():
    html = driver.page_source
    soup = bs4.BeautifulSoup(html,'html.parser')
    links = soup.findAll('div',class_='entry-content')
    episodes = (links[0].findAll('a'))
    links = []
    for episode in episodes:
        link = (str(episode).split('href=')[1].split(' rel')[0]).split('"')[1]
        links.append(link)
    return links

In [8]:
def getDownloadLinks(links : list):
    print(f"Going to fetch {len(links)} links")
    gDriveLinks = {}
    delay = 60 # can wait for max 1 minute to load the website.

    def writeLink(name):
        """This will save all the fetched links"""
        with open(f"links Season {Season}.txt",'a') as f:
            f.write(f"{name} : {gDriveLinks[name]}")
            f.write("\n")

    def extractLink():
        """This method extracts the link"""
        newHtml = driver.page_source
        newSoup = bs4.BeautifulSoup(newHtml,'html.parser')

        # Getting the download button
        newLink = newSoup.findAll('a',class_="btn-primary")

        # Getting the name of the file
        name = newSoup.find('title').text.split('.mkv')[0]

        # Getting it's size
        size = newLink[0].text.split(' [')[1].split(' ]')[0]

        # Getting it's link
        gDriveLink = (newLink[0].get('href'))

        # Adding it to our recors
        gDriveLinks[name] = [gDriveLink,size]

        return name

    for index,link in enumerate(links):
        # Going to the link
        driver.get(link)

        # waiting for the page to load
        try:
            myElem = WebDriverWait(driver, delay).until(EC.element_to_be_clickable((By.ID, 'drc')))
        except TimeoutException:
            print("Loading took too much time!")
        else:
            # going to the next download page
            driver.find_element(By.ID , 'drc').click()

            # waiting for the page to load
            try:
                myElem = WebDriverWait(driver, delay).until(EC.element_to_be_clickable((By.ID, 'wb')))
            except TimeoutException:
                print("Loading took too much time!")
            else:
                # Finally extracting the link
                name = extractLink()
                
                # Writing it to the file
                writeLink(name)
                
                # Displaying the data
                print(gDriveLinks.get(name))


    return gDriveLinks

In [9]:
links = getLinks()

In [78]:
fetched = getDownloadLinks(links)

Going to fetch 32 links
['https://drive.google.com/uc?id=15FjB_eXuxAAsEW7MchFb-Wz-yLzNABVm', ' 128.87MB']
['https://drive.google.com/uc?id=1MEIYgS7LYLgRGowmstPXcN1NtYlAGJOX', ' 118.75MB']
['https://drive.google.com/uc?id=1hulMFCz2k6qgQfvaXeBm8C6o02Nr1PNF', ' 146.54MB']



KeyboardInterrupt

